# 🎯 Structured Output Demo

This notebook demonstrates how to use **structured output** with Fence's Links, Chains, and Agents. Structured output ensures you get properly formatted, typed data instead of raw strings.

## What You'll Learn

- How to define Pydantic models for structured output
- Using structured output with Links
- Chaining structured outputs together  

## Setup

First, let's set up our environment and imports.

In [18]:
from pathlib import Path
current_dir = Path('.').resolve().parents[0]
import sys
sys.path.append(str(current_dir))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from pydantic import BaseModel, Field
from typing import List, Optional
from fence.models.bedrock import Claude35Sonnet
from fence.links import Link
from fence.chains import Chain
from fence.agents.bedrock import BedrockAgent
from fence.templates.string import StringTemplate
from fence.tools.base import BaseTool

## Define Pydantic Models for Structured Output

First, let's define the data structures we want our models to return. These Pydantic models ensure type safety and validation.

In [20]:
class ProductReview(BaseModel):
    """Structured product review analysis."""

    rating: int = Field(..., ge=1, le=5, description="Rating from 1-5 stars")
    sentiment: str = Field(
        ..., description="Overall sentiment: positive, negative, or neutral"
    )
    key_points: List[str] = Field(..., description="Key points mentioned in the review")
    recommendation: str = Field(..., description="Recommendation based on the review")


class CompetitorAnalysis(BaseModel):
    """Structured competitor analysis."""

    competitor_name: str = Field(..., description="Name of the competitor")
    strengths: List[str] = Field(..., description="Competitor's key strengths")
    weaknesses: List[str] = Field(..., description="Competitor's key weaknesses")
    market_position: str = Field(
        ..., description="Market position: leader, challenger, follower, or niche"
    )
    threat_level: int = Field(..., ge=1, le=5, description="Threat level from 1-5")


class MarketInsight(BaseModel):
    """Structured market insight."""

    product_name: str = Field(..., description="Product being analyzed")
    overall_rating: float = Field(..., ge=1.0, le=5.0, description="Average rating")
    total_reviews: int = Field(..., description="Number of reviews analyzed")
    sentiment_summary: str = Field(..., description="Overall sentiment summary")
    top_competitors: List[str] = Field(..., description="Top 3 competitors identified")
    market_recommendation: str = Field(..., description="Strategic recommendation")


print("✅ Pydantic models defined!")
print(f"📋 ProductReview fields: {list(ProductReview.__fields__.keys())}")
print(f"🏢 CompetitorAnalysis fields: {list(CompetitorAnalysis.__fields__.keys())}")
print(f"📊 MarketInsight fields: {list(MarketInsight.__fields__.keys())}")

✅ Pydantic models defined!
📋 ProductReview fields: ['rating', 'sentiment', 'key_points', 'recommendation']
🏢 CompetitorAnalysis fields: ['competitor_name', 'strengths', 'weaknesses', 'market_position', 'threat_level']
📊 MarketInsight fields: ['product_name', 'overall_rating', 'total_reviews', 'sentiment_summary', 'top_competitors', 'market_recommendation']


/var/folders/x0/8q2p5vcd6356_kq623qdnc7h0000gp/T/ipykernel_76510/974015492.py:36: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(f"📋 ProductReview fields: {list(ProductReview.__fields__.keys())}")
/var/folders/x0/8q2p5vcd6356_kq623qdnc7h0000gp/T/ipykernel_76510/974015492.py:37: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(f"🏢 CompetitorAnalysis fields: {list(CompetitorAnalysis.__fields__.keys())}")
/var/folders/x0/8q2p5vcd6356_kq623qdnc7h0000gp/T/ipykernel_76510/974015492.py:38: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed

## Demo 1: Structured Output with Links 🔗

Let's start with the basics - using structured output with individual Links.

In [21]:
# Create a model with structured output
model = Claude35Sonnet(
    source="structured_demo",
    region="us-east-1",
    output_structure=ProductReview,  # 🎯 This is the magic!
)

# Create a link for review analysis
review_analyzer = Link(
    model=model,
    template=StringTemplate(
        """
    Analyze this product review and extract structured information:
    
    Review: "{review_text}"
    
    Please analyze the sentiment, rating, key points, and provide a recommendation.
    """
    ),
    name="review_analyzer",
)

print("🔗 Review analyzer Link created!")
print(f"📝 Model: {model.model_name}")
print(f"🎯 Output Structure: {model.output_structure.__name__}")

🔗 Review analyzer Link created!
📝 Model: Claude 3.5 Sonnet
🎯 Output Structure: ProductReview


In [22]:
# Test with sample reviews
sample_reviews = [
    "This laptop is amazing! Super fast, great battery life, and the screen is gorgeous. Definitely worth the money. 5 stars!",
    "The phone is okay but the battery dies too quickly. Camera is decent though. Would give it 3 stars.",
    "Terrible product. Broke after one week. Customer service was unhelpful. Don't waste your money. 1 star.",
]

for i, review in enumerate(sample_reviews, 1):
    print(f"\n📝 Review {i}:")
    print(f"Text: {review}")

    result = review_analyzer.run(review_text=review)
    result = result[
        "state"
    ]  # 🎯 Dictionary with 'state' key containing the structured data

    print(f"\n✨ Structured Analysis:")
    print(f"   Rating: {result['rating']}/5 ⭐")
    print(f"   Sentiment: {result['sentiment']} 😊")
    print(f"   Key Points: {', '.join(result['key_points'])} 📌")
    print(f"   Recommendation: {result['recommendation']} 💡")


📝 Review 1:
Text: This laptop is amazing! Super fast, great battery life, and the screen is gorgeous. Definitely worth the money. 5 stars!

✨ Structured Analysis:
   Rating: 5/5 ⭐
   Sentiment: positive 😊
   Key Points: Super fast, Great battery life, Gorgeous screen, Worth the money 📌
   Recommendation: Highly recommended for purchase 💡

📝 Review 2:
Text: The phone is okay but the battery dies too quickly. Camera is decent though. Would give it 3 stars.

✨ Structured Analysis:
   Rating: 3/5 ⭐
   Sentiment: neutral 😊
   Key Points: Battery life is poor, Camera is decent, Overall performance is average 📌
   Recommendation: Consider other options if battery life is a priority, but may be suitable for those who prioritize camera quality and don't mind average overall performance. 💡

📝 Review 3:
Text: Terrible product. Broke after one week. Customer service was unhelpful. Don't waste your money. 1 star.

✨ Structured Analysis:
   Rating: 1/5 ⭐
   Sentiment: negative 😊
   Key Points: Prod

## Demo 2: Structured Output with Chains ⛓️

Now let's see how to chain multiple structured outputs together for complex workflows.

In [23]:
# Step 1: Analyze individual reviews
review_model = Claude35Sonnet(
    source="chain_demo", region="us-east-1", output_structure=ProductReview
)

review_link = Link(
    model=review_model,
    template=StringTemplate(
        """
    Analyze this product review:
    "{review_text}"
    """
    ),
    name="review_analyzer",
    output_key="review_analysis",
)

print("🔗 Step 1: Review analyzer created")

🔗 Step 1: Review analyzer created


In [24]:
# Step 2: Analyze competitors mentioned
competitor_model = Claude35Sonnet(
    source="chain_demo", region="us-east-1", output_structure=CompetitorAnalysis
)

competitor_link = Link(
    model=competitor_model,
    template=StringTemplate(
        """
    Based on this review analysis, identify and analyze the main competitor mentioned:
    
    Review Analysis: {review_analysis}
    
    If no specific competitor is mentioned, analyze the general market leader in this category.
    """
    ),
    name="competitor_analyzer",
    output_key="competitor_analysis",
)

print("🏢 Step 2: Competitor analyzer created")

🏢 Step 2: Competitor analyzer created


In [25]:
# Step 3: Generate market insights
insight_model = Claude35Sonnet(
    source="chain_demo", region="us-east-1", output_structure=MarketInsight
)

insight_link = Link(
    model=insight_model,
    template=StringTemplate(
        """
    Generate market insights based on this analysis:
    
    Review Analysis: {review_analysis}
    Competitor Analysis: {competitor_analysis}
    
    Provide strategic recommendations for the product.
    """
    ),
    name="insight_generator",
)

print("📊 Step 3: Market insight generator created")

📊 Step 3: Market insight generator created


In [26]:
# Create the chain
analysis_chain = Chain(links=[review_link, competitor_link, insight_link])

print("⛓️ Analysis chain created with 3 structured output steps!")
print(f"🔗 Links: {[link.name for link in analysis_chain.links]}")

⛓️ Analysis chain created with 3 structured output steps!
🔗 Links: ['review_analyzer', 'competitor_analyzer', 'insight_generator']


In [ ]:
# Run the chain with a sample review
sample_review = """
I switched from iPhone to this Samsung Galaxy and I'm impressed! 
The camera is better than my old iPhone 13, battery lasts all day, 
and the price was much more reasonable. The only downside is some 
apps work better on iOS, but overall very happy. 4 stars!
"""

print(f"📱 Analyzing Review: {sample_review.strip()[:100]}...")
print("\n🔄 Running chain...")

result = analysis_chain.run(input_dict={"review_text": sample_review})
final_insight = result["state"]

print(f"\n🎯 Market Insight Generated:")
print(f"   📱 Product: {final_insight['product_name']}")
print(f"   ⭐ Overall Rating: {final_insight['overall_rating']}/5.0")
print(f"   😊 Sentiment: {final_insight['sentiment_summary']}")
print(f"   🏢 Top Competitors: {', '.join(final_insight['top_competitors'])}")
print(f"   💡 Recommendation: {final_insight['market_recommendation']}")
print(f"   📊 Total Reviews Analyzed: {final_insight['total_reviews']}")

📱 Analyzing Review: I switched from iPhone to this Samsung Galaxy and I'm impressed! 
The camera is better than my old i...

🔄 Running chain...
